## Recurrent Layers


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### RNN

In [2]:
"""
nn.RNN(
    input_size=Hin,
    hidden_size=Hout,
    num_layers=C=1,
    batch_first={(L, N?): False, (N?, L): True}[B],
    bidirectional={1: False, 2: True}[D]
):  (*B, Hin),    (D*C, N?, Hout)
->  (*B, D*Hout), (D*C, N?, Hout)
"""

# https://docs.pytorch.org/docs/stable/generated/torch.nn.RNN.html
# https://karpathy.github.io/2015/05/21/rnn-effectiveness/
"""Define Layer"""

"""1. Position Arguments"""
input_size, hidden_size, num_layers, nonlinearity, bias = 2, 3, 4, "tanh", True

"""2. Keyword Arguments"""
batch_first, bidirectional = False, False

rnn = nn.RNN(
    input_size,  # Hin, Required
    hidden_size,  # Hout, Required
    num_layers,  # C, default=1
    nonlinearity,  # "tanh" or "relu", default="tanh"
    bias,  # default=True
    batch_first=batch_first,  # default=False
    dropout=0.0,
    bidirectional=bidirectional,  # default=False
    device=None,
    dtype=None,
)

D = 2 if bidirectional else 1

"""Forward Pass"""

"""1. Inputs"""
batch_size, seq_len = 5, 6

if batch_first:
    x = torch.randn(batch_size, seq_len, input_size)  # (N, L, Hin)
else:
    x = torch.randn(seq_len, batch_size, input_size)  # (L, N, Hin)

h0 = torch.randn(D * num_layers, batch_size, hidden_size)  # (D * C, N, Hout)


"""2. Outputs"""
y, h = rnn(x, None if h0 is None else h0)

if batch_first:
    assert y.shape == (batch_size, seq_len, D * hidden_size)  # (N, L, D * Hout)
else:
    assert y.shape == (seq_len, batch_size, D * hidden_size)  # (L, N, D * Hout)

assert h.shape == (D * num_layers, batch_size, hidden_size)  # (D * C, N, Hout)

### RNNCell

In [3]:
"""
nn.RNNCell(
    input_size=Hin,
    hidden_size=Hout,
):  (N?, Hin),  (N?, Hout)
->  (N?, Hout), (N?, Hout)
"""
# https://docs.pytorch.org/docs/stable/generated/torch.nn.RNNCell.html
"""Define Layer"""

"""1. Position Arguments"""
input_size, hidden_size, bias, nonlinearity = 2, 3, True, "tanh"


rnn_cell = nn.RNNCell(
    input_size,  # Hin, Required
    hidden_size,  # Hout, Required
    bias,  # default=True
    nonlinearity,  # "tanh" or "relu", default="tanh"
    device=None,
    dtype=None,
)

f = {"tanh": F.tanh, "relu": F.relu}[nonlinearity]

"""Forward Pass"""

"""1. Inputs"""
batch_size = 4

x = torch.randn(batch_size, input_size)  # (N, Hin)

h0 = torch.randn(batch_size, hidden_size)  # (N, Hout)


"""2. Outputs"""
h = rnn_cell(x, None if h0 is None else h0)

assert h.shape == (batch_size, hidden_size)  # (N, Hout)
assert torch.allclose(
    h,
    f(
        F.linear(x, rnn_cell.weight_ih, rnn_cell.bias_ih)
        + F.linear(h0, rnn_cell.weight_hh, rnn_cell.bias_hh)
    ),
    atol=1e-6,
)

### LSTM

In [4]:
"""
nn.LSTM(
    input_size=Hin,
    hidden_size=Hout,
    num_layers=C=1,
    batch_first={(L, N?): False, (N?, L): True}[B],
    bidirectional={1: False, 2: True}[D],
    proj_size=P=P if P > 0 else Hout
):  (*B, Hin), ((D*C, N?, P), (D*C, N?, Hout))
->  (*B, D*P), ((D*C, N?, P), (D*C, N?, Hout))
"""
# https://docs.pytorch.org/docs/stable/generated/torch.nn.LSTM.html
# https://colah.github.io/posts/2015-08-Understanding-LSTMs/
"""Define Layer"""

"""1. Position Arguments"""
input_size, hidden_size, num_layers, bias = 2, 3, 1, True

"""2. Keyword Arguments"""
batch_first, bidirectional, proj_size = False, True, 1

assert proj_size < hidden_size, "proj_size has to be smaller than hidden_size"

lstm = nn.LSTM(
    input_size,  # Hin
    hidden_size,  # Hout
    num_layers,  # C
    bias=bias,
    batch_first=batch_first,
    dropout=0.0,
    bidirectional=bidirectional,
    proj_size=proj_size,  # default=0
    device=None,
    dtype=None,
)

D = 2 if bidirectional else 1
P = proj_size if proj_size > 0 else hidden_size

"""Forward Pass"""

"""1. Inputs"""
batch_size, seq_len = 4, 5

if batch_first:
    x = torch.randn(batch_size, seq_len, input_size)  # (N, L, Hin)
else:
    x = torch.randn(seq_len, batch_size, input_size)  # (L, N, Hin)

h = torch.randn(D * num_layers, batch_size, P)  # (D * C, N, P)
c = torch.randn(D * num_layers, batch_size, hidden_size)  # (D * C, N, Hout)


"""2. Outputs"""
y, (h, c) = lstm(x, None if h is None or c is None else (h, c))

if batch_first:
    assert y.shape == (batch_size, seq_len, D * P)  # (N, L, D * P)
else:
    assert y.shape == (seq_len, batch_size, D * P)  # (L, N, D * P)

assert h.shape == (D * num_layers, batch_size, P)  # (D * C, N, P)
assert c.shape == (D * num_layers, batch_size, hidden_size)  # (D * C, N, Hout)

### LSTMCell

In [5]:
"""
nn.LSTMCell(
    input_size=Hin,
    hidden_size=Hout,
):  (N?, Hin), ((N?, P), (N?, Hout))
->  (N?, P),   ((N?, P), (N?, Hout))
"""
# https://docs.pytorch.org/docs/stable/generated/torch.nn.LSTMCell.html
# https://yb.tencent.com/s/4OGnkvsVzqDH
"""Define Layer"""

"""1. Position Arguments"""
input_size, hidden_size, bias = 2, 3, True

lstm_cell = nn.LSTMCell(
    input_size,  # Hin
    hidden_size,  # Hout
    bias,
    device=None,
    dtype=None,
)

"""Forward Pass"""

"""1. Inputs"""
batch_size = 4

x = torch.randn(batch_size, input_size)  # (N, Hin)

h0 = torch.randn(batch_size, hidden_size)  # (D * C, N, Hout)
c0 = torch.randn(batch_size, hidden_size)  # (D * C, N, Hout)


"""2. Outputs"""
h, c = lstm_cell(x, None if h0 is None or c0 is None else (h0, c0))

assert h.shape == (batch_size, hidden_size)  # (N, Hout)
assert c.shape == (batch_size, hidden_size)  # (N, Hout)

gates = F.linear(x, lstm_cell.weight_ih, lstm_cell.bias_ih) + F.linear(
    h0, lstm_cell.weight_hh, lstm_cell.bias_hh
)
i, f, g, o = gates.chunk(4, dim=1)  # Split into input, forget, gate and output
i = F.sigmoid(i)
f = F.sigmoid(f)
g = F.tanh(g)
o = F.sigmoid(o)

c1 = f * c0 + i * g  # Update cell state
h1 = o * F.tanh(c)  # Update hidden state

assert torch.allclose(h, h1, atol=1e-6)
assert torch.allclose(c, c1, atol=1e-6)

### GRU

In [6]:
"""
nn.GRU(
    input_size=Hin,
    hidden_size=Hout,
    num_layers=C=1,
    batch_first={(L, N?): False, (N?, L): True}[B],
    bidirectional={1: False, 2: True}[D]
):  (*B, Hin),    (D*C, N?, Hout)
->  (*B, D*Hout), (D*C, N?, Hout)
"""
# https://docs.pytorch.org/docs/stable/generated/torch.nn.GRU.html
"""Define Layer"""

"""1. Position Arguments"""
input_size, hidden_size, num_layers, bias = 2, 3, 1, True

"""2. Keyword Arguments"""
batch_first, bidirectional = False, True

gru = nn.GRU(
    input_size,  # Hin
    hidden_size,  # Hout
    num_layers,  # C
    bias,
    batch_first=batch_first,
    dropout=0.0,
    bidirectional=bidirectional,
    device=None,
    dtype=None,
)

D = 2 if bidirectional else 1

"""Forward Pass"""

"""1. Inputs"""
batch_size, seq_len = 4, 5

if batch_first:
    x = torch.randn(batch_size, seq_len, input_size)  # (N, L, Hin)
else:
    x = torch.randn(seq_len, batch_size, input_size)  # (L, N, Hin)

h = torch.randn(D * num_layers, batch_size, hidden_size)  # (D * C, N, Hout)


"""2. Outputs"""
y, h = gru(x, None if h is None else h)

if batch_first:
    assert y.shape == (batch_size, seq_len, D * hidden_size)  # (N, L, D * Hout)
else:
    assert y.shape == (seq_len, batch_size, D * hidden_size)  # (L, N, D * Hout)

assert h.shape == (D * num_layers, batch_size, hidden_size)  # (D * C, N, Hout)

### GRUCell

In [7]:
"""
nn.GRUCell(
    input_size=Hin,
    hidden_size=Hout,
):  (N?, Hin),  (N?, Hout)
->  (N?, Hout), (N?, Hout)
"""
# https://docs.pytorch.org/docs/stable/generated/torch.nn.GRUCell.html
"""Define Layer"""

"""1. Position Arguments"""
input_size, hidden_size, bias = 2, 3, True

gru_cell = nn.GRUCell(
    input_size,  # Hin
    hidden_size,  # Hout
    bias=bias,
    device=None,
    dtype=None,
)

"""Forward Pass"""

"""1. Inputs"""
batch_size = 4

x0 = torch.randn(batch_size, input_size)  # (N, Hin)

h0 = torch.randn(batch_size, hidden_size)  # (N, Hout)


"""2. Outputs"""
h = gru_cell(x0, None if h0 is None else h0)

assert h.shape == (batch_size, hidden_size)  # (N, Hout)

x_gates = F.linear(x0, gru_cell.weight_ih, gru_cell.bias_ih).chunk(3, dim=1)
h_gates = F.linear(h0, gru_cell.weight_hh, gru_cell.bias_hh).chunk(3, dim=1)
r = F.sigmoid(x_gates[0] + h_gates[0])  # Reset gate
z = F.sigmoid(x_gates[1] + h_gates[1])  # Update gate
n = F.tanh(x_gates[2] + r * h_gates[2])  # New gate
h1 = (torch.ones_like(z) - z) * n + z * h0  # Update hidden state

assert torch.allclose(h, h1, atol=1e-6)